In [ ]:
import sys
!{sys.executable} -m pip install kfp==1.4.0 kfp-server-api==1.2.0 --user>/dev/null

In [ ]:
import os,sys
import kfp
import random
import string

In [ ]:
components_url = "/mnt/dkube/pipeline/components/"
dkube_training_op = kfp.components.load_component_from_file(components_url + "training/component.yaml")
dkube_serving_op = kfp.components.load_component_from_file(components_url + "serving/component.yaml")
token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(host=os.getenv("KF_PIPELINES_ENDPOINT"), existing_token=token, namespace=os.getenv("USERNAME"))

In [ ]:
@kfp.dsl.pipeline(
    name='dkube-mnist-pl',
    description='sample mnist pipeline with dkube components'
)
def mnist_pipeline():
    train       = dkube_training_op(token,container = '{"image":"ocdr/d3-datascience-pytorch-cpu:v1.6"}',
                                    framework="pytorch", version="1.6",
                                    program="pytorch-examples", run_script="python mnist/train.py",
                                    datasets='["mnist"]', outputs='["mnist"]',
                                    input_dataset_mounts='["/mnist"]',
                                    output_mounts='["/model"]',
                                    envs='[{"EPOCHS": "1"}]')

    serving     = dkube_serving_op(token,model = train.outputs['artifact'], device='cpu', 
                                    serving_image='{"image":"ocdr/pytorchserver:1.6"}',
                                    transformer_image='{"image":"ocdr/d3-datascience-pytorch-cpu:v1.6-1"}',
                                    transformer_project="pytorch-examples",
                                    transformer_code='mnist/transformer.py').after(train)

In [ ]:
client.create_run_from_pipeline_func(mnist_pipeline, arguments={})

In [ ]:
# compile
import tempfile
f = tempfile.NamedTemporaryFile(suffix=".zip", delete=False)
kfp.compiler.Compiler().compile(mnist_pipeline,f.name)

In [ ]:
pipeline_id = client.get_pipeline_id("mnist-pipeline")

if(pipeline_id != None):
    pipeline_version = ''.join(random.choices(string.ascii_uppercase + string.digits, k = 7))
    new_pipeline = client.upload_pipeline_version(f.name,pipeline_version_name = pipeline_version, pipeline_id = pipeline_id)
else:
    pipeline = client.upload_pipeline(f.name, pipeline_name="mnist-pipeline")
